In [1]:
import random
def generate_reads(seq, k, min_overlap, max_overlap,seed=None):
    random.seed(seed)
    reads = []
    for i in range(0, len(seq)-k+1):
        # Generate a random overlap within the specified range
        overlap = random.randint(min_overlap, max_overlap)
        start = i
        end = i + k + overlap
        reads.append(seq[start:end])
    return reads

def generate_genome_sequence(n,seed=None):
    random.seed(seed)
    nucleotides = {1:'A',2:'C',3:'G',4:'T'}
    seq = ''
    for i in range(n):
        seq += nucleotides[random.randint(1,4)]
    return seq

In [2]:
'''
DESCRIPTION
INPUT
OUTPUT
'''
class Stalk:
    def __init__(self,stalk):
        self.stalk = stalk
        if len(self.stalk) == 0: self.stalk = '$'

    def __repr__(self): return self.stalk

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.stalk[index]

    def __len__(self):
        if self.stalk == '$' or self.stalk == '^': return 0
        return len(self.stalk)
    
    def __str__(self):
        if self.stalk =='$': return ''
        else: return self.stalk

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def common_substring(self,other):
        i = 0
        substr = ''
        if type(other) == str: other = Stalk(other)
        while i < min(len(self),len(other)) and self[i] == other[i]:
            substr += self[i]
            i += 1
        return Stalk(substr), Stalk(self[i:]), Stalk(other[i:])

In [3]:
'''
DESCRIPTION
INPUT
OUTPUT
'''
class Leaf:
    def __init__(self,left,right=''):
        if len(left) == 0:
            self.left = left
            self.right = 1
        else:
            self.left = left
            self.right = Leaf(right)
    
    def __repr__(self): return str(self.right)

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self.left)

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)
    
    def __is_shallow__(self): return True

In [4]:
class Sequence:
    def __init__(self,seq=None,seen=None):
        self.extensions = []
        self.contains = set()
        if seq is None: 
            self.seq = ''
            self.seen = tuple()
        else: 
            self.seq = seq
            if seen is None: self.seen = (seq,)
            else: self.seen = seen

    def __repr__(self): return self.seq

    def copy(self):
        c = Sequence()
        c.extensions = self.extensions[:]
        c.contains = self.contains.copy()
        c.seen = self.seen[:]
        c.seq = self.seq
        return c 

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def add(self,item): 
        if item not in self.seen: self.seen = tuple(list(self.seen) + [item]) 

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def __add__(self,other):
        self.seq += other.seq
        self.seen = tuple(list(self.seen)+list(other.seen))
        for a in set(self.extensions).intersection(self.seen):
            self.extensions.pop(a)
        for a in set(other.extensions).intersection(self.seen):
            other.extensions.pop(a)
        self.extensions += other.extensions
        return self

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''    
    def rollback(self):
        self.seen = self.seen[:-1]
        self.seq = self.seq[:self.seq.find(self.seen[-1])+len(self.seen[-1])]
        contains = set()
        for read in self.contains:
            if read in self.seq: contains.add(read)
        self.contains.clear()
        self.contains.update(contains)

In [5]:
'''
DESCRIPTION
INPUT
OUTPUT
    '''
class Branch:
    def __init__(self):
        self.b = {}
        self.s = {}

    def __repr__(self): return repr(self.b)

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index):
        if type(index) == str: return self.b[Stalk(index)]
        return self.b[index]

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def __is_shallow__(self):
        for a in self.b.values():
            if type(a) == Branch: return False
        return True

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def __traverse__(self,context):
        b = self[context[0]]
        s = self.s[context[0]]
        context = context[len(s[0]):]
        while len(context) > 0 and len(b) > 1 and context[0] in b.s:
            s = b.s[context[0]]
            b = b[context[0]]
            context = context[len(s[0]):]
        return b
    
    def __setitem__(self,index,value):
        if type(index) == str: self.b[Stalk(index)] = value
        else: self.b[index] = value

    def __contains__(self,other): 
        if type(other) == str: return Stalk(other) in self.b
        return other in self.b

    def __len__(self): return len(self.b)

    def pop(self,index): return self.b.pop(index)

    '''
    DESCRIPTION
        adds a suffix to the trie
    INPUT
        stalk | a Stalk() which is a common substring of every read up to this point and beyond
        reads | a set of reads which have with the same common substring up to this point
    '''
    def add(self,stalk,reads):
        if stalk in self:
            if not len(stalk):
                self[stalk].right+=1
                self.s[stalk][1].update(reads)
                return
            if type(self[stalk]) == Leaf:
                branch = Branch()
                l1 = self.pop(stalk)
                stalk_ = list(self.s.pop(stalk))
                stalk_[0],l1.left,l2 = stalk_[0].common_substring(stalk)
                for read in reads:
                    # if str(stalk_[0]) in read and str(stalk_[0]) != '$': stalk_[1].add(read)
                    if read.endswith(str(stalk_[0])): stalk_[1].add(read)
                branch.add(l1.left,stalk_[1].copy())
                branch.add(l2,reads)
                stalk_ = tuple(stalk_)
                self[stalk_[0]] = branch
                self.s[stalk_[0]] = stalk_
            else:
                stalk_ = list(self.s.pop(stalk))
                branch = self.pop(stalk)
                stalk_[0],bstalk,stalk = stalk_[0].common_substring(stalk)
                if len(bstalk):
                    br = Branch()
                    br[bstalk] = branch 
                    br.s[bstalk] = (bstalk,stalk_[1].copy())
                    br.add(stalk,reads)
                    self[stalk_[0]] = br
                else: 
                    branch.add(stalk,reads)
                stalk_[1].update(reads)
                stalk_ = tuple(stalk_)
                if not len(bstalk): self[stalk_[0]] = branch
                self.s[stalk_[0]] = stalk_
        else:
            if type(stalk) == str: stalk = Stalk(stalk)
            self.s[stalk] = (stalk,reads)
            self[stalk] = Leaf(stalk)

    '''
    DESCRIPTION
        a method which returns all reads beyond a certain point on a branch
    INPUT
        exclude    | a list of all reads which should not be added as an extension
        context    | the path from the root up to that point in the branch
        t          | a string which holds the path to the read from the context
    OUTPUT
        extensions | a dictionary of all possible extensions indexed on the read and pointing
                   | to a 3-tuple holding the context, a Sequence() of the read along the path
                   | a Sequence() of the read after the context 
    '''
    def extensions(self,exclude,context):
        # if t is None: t = ''
        extensions = []
        exclusions = set()
        for s in self.s:
            if type(self[s]) is Branch:
                for b in self[s].b:
                    if type(self[s].b[b]) is Branch:
                        ext, exc =  self[s].b[b].extensions(exclude,context)
                        extensions += ext
                        exclusions.update(exc)
                        continue
                    for read in self[s].s[self[s].b[b].left][1]:
                        if read in exclude: 
                            exclusions.add(read)
                            continue
                        t = read
                        for _ in range(read.count(context)):
                            t = t.partition(context)[2]
                            extensions += [(context,\
                                            Sequence(read),\
                                            Sequence(t,(read,)))]
            else:
                for read in self.s[s][1]: 
                    if read in exclude: 
                        exclusions.add(read)
                        continue
                    t = read
                    for _ in range(read.count(context)):
                        t = t.partition(context)[2]
                        extensions += [(context,\
                            Sequence(read),\
                            Sequence(t,(read,)))]
        return extensions, exclusions

In [6]:
'''
DESCRIPTION
    an object which constructs a suffix trie out of fragments of a sequence and can traverse 
    the trie to resconstruct some target sequence
INPUT
    reads | a list of strings which overlap and are fragments of a longer sequence
'''
class Sequitur:
    def __init__(self,reads,correct_sequence=None):
        if correct_sequence is not None: self.correct_sequence = correct_sequence
        self.branch = Branch()
        self.reads = reads
        for read in reads:
            for i in range(len(read)): 
                self.branch.add(Stalk(read[i:]),{read})
        self.sequence = Sequence()

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def context_depth_sort(self,e): return e[1].seq.find(e[0])

    '''
    DESCRIPTION
        method that attempts to extend a sequence or join a prefix to a suffix
    INPUT
        sequence | a Sequence() with the extension read
        prefix   | a Sequence() that has been previously extended with the prefix read
        suffix   | a Sequence() that has reached the terminus
    OUTPUT
        whether or not sequence was successfully extended
    '''
    # def extend(self,proto,file):
    #     extensions = list(proto.extensions.values())
    #     extensions.sort(key=self.context_depth_sort,reverse=True)
    #     for extension in extensions:
    #         file.write(proto.seq+","+\
    #                     str(proto.seen)+","+\
    #                     str(proto.contains)+","+\
    #                     extension[0]+","+\
    #                     extension[1].seq+","+\
    #                     extension[2].seq+","+\
    #                     str(self.sequitur(proto+extension[2],file=file))+"\n")

    '''
    DESCRIPTION
        a method which traverses a suffix trie and attempts to reconstruct the sequence from which the 
        trie was constructed
    INPUT
        sequence | a Sequence() with a read chosen as an initus
        prefix   | an optional Sequence() which has been extended but has not reached the terminus. 
                 | defaults to and empty Sequence()
        suffix   | an optional Sequence() which has reached the terminus. defaults to an empty 
                 | Sequence().
        seed     | an optional integer to seed the randomiser. defaults to None.
    '''
    def sequitur(self,proto,seed=None,df=None):
        import random
        if seed is not None: random.seed(seed)
        if not len(set(self.reads).difference(proto.contains.union(proto.seen))): return
        self.sequence = proto.copy()

        i = 1
        context = self.sequence.seq[-i:]
        seen = self.sequence.contains\
                        .union(self.sequence.seen)\
                        .union(proto.contains\
                        .union(proto.seen))
        branch = self.branch.__traverse__(context)
        self.sequence.extensions.clear()
        extensions,exclusions = branch.extensions(seen,context)
        while i < len(self.sequence.seq) and not branch.__is_shallow__():
            i += 1
            context = self.sequence.seq[-i:]
            branch = self.branch.__traverse__(context)
            if type(branch) is Branch: extensions,exclusions = branch.extensions(seen,context)
            if type(branch) is Leaf or not len(extensions):
                i -= 1
                context = self.sequence.seq[-i:]
                branch = self.branch.__traverse__(context)
                extensions,exclusions = branch.extensions(seen,context)
                break
        self.sequence.extensions += extensions
        extensions = self.sequence.extensions
        extensions.sort(key=self.context_depth_sort,reverse=True)
        while i > 1:
            for extension in extensions:
                df.loc[len(df.index)] = [self.sequence.seq, #proto_sequence
                                         extension[1].seq, # extension
                                        len(extension[0]), # common_string_length
                                        self.sequence.seq.count(extension[0]), # common_string_count
                                        len(extension[1].seq[:extension[1].seq.find(extension[0])+len(extension[0])]), # overlap_string_length
                                        self.sequence.seq.count(extension[1].seq[:extension[1].seq.find(extension[0])+len(extension[0])]), # overlap_string_count
                                        len(extension[2].seq),# extension_addition_length
                                        str(self.sequence.seq.endswith(extension[1].seq[:extension[1].seq.find(extension[0])+len(extension[0])])), # valid_overlap
                                        str(self.sequence.seq.endswith(extension[1].seq[:extension[1].seq.find(extension[0])+len(extension[0])]) and\
                                             self.sequence.seq+extension[2].seq in self.correct_sequence)] # valid_extension
                self.sequitur(self.sequence+extension[2],seed,df)
                self.sequence = proto.copy()
            i -= 1
            context = self.sequence.seq[-i:]
            branch = self.branch.__traverse__(context)
            extensions,exclusions = branch.extensions(seen,context)
            self.sequence.extensions.clear()
            self.sequence.extensions += extensions
            extensions = self.sequence.extensions
            extensions.sort(key=self.context_depth_sort,reverse=True)

In [7]:
import pandas as pd

df = pd.DataFrame(columns=['proto_sequence',
                           'extension',
                           'common_string_length',
                           'common_string_count',
                           'overlap_string_length',
                           'overlap_string_count',
                           'extension_addition_length',
                           'valid_overlap',
                           'valid_extension'])

In [9]:
seed = 0
# sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
# reads = ['betty_bought_butter_th',
#                         'tter_the_butter_was_',
#                               'he_butter_was_bitter_',
#                                          'as_bitter_betty_bought',
#                                                      'tty_bought_better_butter_t',
#                                                                      'r_butter_to_make_the_',
#                                                                                    'ke_the_bitter_butter_better']
# sequence = 'you say hello world, i bellow go to hell'
# reads = ['you say hel',
#             ' say hello wo',
#                     'lo world, i be',
#                           'ld, i bellow go t',
#                                     'ow go to hell']
sequence = 'she_sells_sea_shells_on_the_sea_shore'
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
sequitur = Sequitur(reads,sequence)
for read in reads:
    sequitur.sequitur(Sequence(read),seed=seed,df=df)

In [ ]:
sequitur.sequitur(Sequence(reads[0]),seed=seed,df=df)
# sequitur.sequitur(Sequence(reads[1]),seed=seed,df=df)
# sequitur.sequitur(Sequence(reads[2]),seed=seed,df=df)
# sequitur.sequitur(Sequence(reads[3]),seed=seed,df=df)
# sequitur.sequitur(Sequence(reads[4]),seed=seed,df=df)
# sequitur.sequitur(Sequence(reads[5]),seed=seed,df=df)
# sequitur.sequitur(Sequence(reads[6]),seed=seed,df=df)

In [54]:
df.loc[:,'common_string_length':'valid_overlap'].astype({'common_string_length':'int',
                                                        'common_string_count':'int',
                                                        'overlap_string_length':'int',
                                                        'overlap_string_count':'int',
                                                        'extension_addition_length':'int',
                                                        'valid_overlap':'bool'}).dtypes

common_string_length         int32
common_string_count          int32
overlap_string_length        int32
overlap_string_count         int32
extension_addition_length    int32
valid_overlap                 bool
dtype: object

In [63]:
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
import numpy as np

X = df.loc[:,'common_string_length':'valid_overlap'].astype({'common_string_length':'int',
                                                        'common_string_count':'int',
                                                        'overlap_string_length':'int',
                                                        'overlap_string_count':'int',
                                                        'extension_addition_length':'int',
                                                        'valid_overlap':'bool'}) # columns with attributes
Y = df.loc[:,'valid_extension'] # column with class labe

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

decision_tree = tree.DecisionTreeClassifier().fit(X_train,Y_train)

# plt.figure(figsize=(50,50))
# tree.plot_tree(decision_tree,
#                 feature_names=['common_string_length',
#                            'common_string_count',
#                            'overlap_string_length',
#                            'overlap_string_count',
#                            'extension_addition_length',
#                            'valid_overlap'],
#                 class_names=['True','False'],
#                 filled=True
# )
# plt.show()

In [79]:
np.array((list(decision_tree.predict(X_test)),Y_test)).where()

array([['False', 'False', 'True', ..., 'False', 'False', 'False'],
       ['False', 'False', 'True', ..., 'False', 'True', 'False']],
      dtype=object)

In [80]:
decision_tree.predict(X_test)

array(['False', 'False', 'True', ..., 'False', 'False', 'False'],
      dtype=object)

In [81]:
X_test

,common_string_length,common_string_count,overlap_string_length,overlap_string_count,extension_addition_length,valid_overlap
5512,5,1,7,0,4,True
3319,2,3,4,1,4,True
2051,2,4,4,2,4,True
3014,2,3,4,1,4,True
2896,2,4,4,1,4,True
...,...,...,...,...,...,...
6095,2,3,4,1,4,True
5426,4,2,7,1,4,True
7685,2,1,7,1,4,True
145,2,5,4,2,4,True


In [10]:
df

,proto_sequence,extension,common_string_length,common_string_count,overlap_string_length,overlap_string_count,extension_addition_length,valid_overlap,valid_extension
0,you say hel,say hello wo,5,1,8,1,5,True,True
1,you say hello wo,"lo world, i be",2,1,5,1,9,True,True
2,"you say hello world, i be","ld, i bellow go t",2,1,8,1,9,True,True
3,you say hel,ow go to hell,4,1,12,0,1,False,False
4,you say hell,say hello wo,4,1,9,1,4,True,True
...,...,...,...,...,...,...,...,...,...
7704,he_sea_shells_on_the_s,ea_shore,2,3,4,1,4,False,False
7705,he_sea_shells_on_the_s,ea_shells_o,2,3,4,1,7,False,False
7706,he_sea_shells_on_the_s,ea_shore,2,3,4,1,4,False,False
7707,he_sea_s,ea_shells_o,2,2,4,1,7,True,False


In [ ]:
seed = 5
sequence = generate_genome_sequence(200,seed=seed)
reads = generate_reads(sequence,3,4,10,seed=seed)
sequitur = Sequitur(reads,sequence)

In [ ]:
# i = 0
# for read in reads:
sequitur.sequitur(Sequence(reads[47]),seed=seed)
print(47,sequitur.sequence.seq ==  sequence)
    # i+=1

In [ ]:
sequence = 'you say hello world, i bellow go to hell'
reads = ['you say hel',
            ' say hello wo',
                    'lo world, i be',
                          'ld, i bellow go t',
                                    'ow go to hell']
sequitur = Sequitur(reads,sequence)
successes = 0
n = 200
for seed in range(n):    
    for read in reads:
        s = 'Seed: ' + str(seed) + ' | Initus: ' + read + ' | '
        sequitur.sequitur(Sequence(read),seed=seed)
        if sequitur.sequence.seq == sequence: 
            s+='SUCCESS'
            successes+=1
        else: s+='FAILURE'
        print(s + ' | ' + sequitur.sequence.seq)
        print('-----------------------------------------')
print('ACCURACY: '+str((successes/(200*len(reads)))*100)+'%')

Seed: 0 | Initus: you say hel | SUCCESS | you say hello world, i bellow go to hell
-----------------------------------------
Seed: 0 | Initus:  say hello wo | SUCCESS | you say hello world, i bellow go to hell
-----------------------------------------
Seed: 0 | Initus: lo world, i be | SUCCESS | you say hello world, i bellow go to hell
-----------------------------------------
Seed: 0 | Initus: ld, i bellow go t | SUCCESS | you say hello world, i bellow go to hell
-----------------------------------------
Seed: 0 | Initus: ow go to hell | SUCCESS | you say hello world, i bellow go to hell
-----------------------------------------
Seed: 1 | Initus: you say hel | SUCCESS | you say hello world, i bellow go to hell
-----------------------------------------
Seed: 1 | Initus:  say hello wo | SUCCESS | you say hello world, i bellow go to hell
-----------------------------------------
Seed: 1 | Initus: lo world, i be | SUCCESS | you say hello world, i bellow go to hell
------------------------

In [ ]:
sequence = 'she_sells_sea_shells_on_the_sea_shore'
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
sequitur = Sequitur(reads)
successes = 0
n = 200
for seed in range(n):    
    for read in reads:
        s = 'Seed: ' + str(seed) + ' | Initus: ' + read + ' | '
        sequitur.sequitur(Sequence(read),seed=seed)
        if sequitur.sequence.seq == sequence: 
            s+='SUCCESS'
            successes+=1
        else: s+='FAILURE'
        print(s + ' | ' + sequitur.sequence.seq)
        print('-----------------------------------------')
print('ACCURACY: '+str((successes/(n*len(reads)))*100)+'%')

Seed: 0 | Initus: she_sells_s | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------
Seed: 0 | Initus: lls_sea_shel | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------

Seed: 0 | Initus: ea_shells_o | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------
Seed: 0 | Initus: shells_on_the_s | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------
Seed: 0 | Initus: he_sea_s | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------
Seed: 0 | Initus: ea_shore | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------
Seed: 1 | Initus: she_sells_s | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------
Seed: 1 | Initus: lls_sea_shel | SUCCESS | she_sells_sea_shells_on_the_sea_shore
-----------------------------------------

Seed: 1 | Initus: ea_s

In [ ]:
sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']
sequitur = Sequitur(reads,sequence)
successes = 0
n = 200
for seed in range(n):    
    for read in reads:
        s = 'Seed: ' + str(seed) + ' | Initus: ' + read + ' | '
        sequitur.sequitur(Sequence(read),seed=seed)
        if sequitur.sequence.seq == sequence: 
            s+='SUCCESS'
            successes+=1
        else: s+='FAILURE'
        print(s + ' | ' + sequitur.sequence.seq)
        print('-----------------------------------------')
print('ACCURACY: '+str((successes/(n*len(reads)))*100)+'%')

Seed: 0 | Initus: betty_bought_butter_th | SUCCESS | betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better
-----------------------------------------



IndexError: tuple index out of range

In [ ]:
successes = 0
n = 200
for seed in range(n):   
    sequence = generate_genome_sequence(200,seed=seed)
    reads = generate_reads(sequence,3,4,10,seed=seed)
    sequitur = Sequitur(reads,sequence)
    for read in reads:
        s = 'Seed: ' + str(seed) + ' | Initus: ' + read + ' | '
        sequitur.sequitur(Sequence(read),seed=seed)
        if sequitur.sequence.seq == sequence: 
            s+='SUCCESS'
            successes+=1
        else: s+='FAILURE'
        print(s + ' | ' + sequitur.sequence.seq)
        print('-----------------------------------------')
print('ACCURACY: '+str((successes/(n*len(reads)))*100)+'%')

Seed: 0 | Initus: TTAGTTGTGCCGC | SUCCESS | TTAGTTGTGCCGCAGCGAAGTAGTGCTTGAAATATGCGACCCCTAAGTAGGAGCGTATGCGCCCAGTAACCAATGCCTGTTGAGATGCCAGACGCGTAACCAAAACATAGAAACCATCAATAGACAGGTCATAATCGGTCCACCGGATCATTGGTGCATAGAGCCTGGGCGTTAACGCCCTTTATTACTAGCTTAATGGT
-----------------------------------------
Seed: 0 | Initus: TAGTTGTGCC | SUCCESS | TTAGTTGTGCCGCAGCGAAGTAGTGCTTGAAATATGCGACCCCTAAGTAGGAGCGTATGCGCCCAGTAACCAATGCCTGTTGAGATGCCAGACGCGTAACCAAAACATAGAAACCATCAATAGACAGGTCATAATCGGTCCACCGGATCATTGGTGCATAGAGCCTGGGCGTTAACGCCCTTTATTACTAGCTTAATGGT
-----------------------------------------
Seed: 0 | Initus: AGTTGTGCCGCAG | SUCCESS | TTAGTTGTGCCGCAGCGAAGTAGTGCTTGAAATATGCGACCCCTAAGTAGGAGCGTATGCGCCCAGTAACCAATGCCTGTTGAGATGCCAGACGCGTAACCAAAACATAGAAACCATCAATAGACAGGTCATAATCGGTCCACCGGATCATTGGTGCATAGAGCCTGGGCGTTAACGCCCTTTATTACTAGCTTAATGGT
-----------------------------------------
Seed: 0 | Initus: GTTGTGCCGC | SUCCESS | TTAGTTGTGCCGCAGCGAAGTAGTGCTTGAAATATGCGACCCCTAAGTAGGAGCGTATGCGCCCAGTAACCAATGCCTGTTGAGATGCCAGACGCGTAACC

IndexError: tuple index out of range